# CAS API Substance Module

The CAS API supports workflow integration, chemical research, machine learning, and cheminformatics.  Learn more about workflow integration solutions from __[CAS Custom Services℠](https://www.cas.org/cas-custom-services).__

This documentation is provided as a demonstration of some of the capabilities and endpoints available in the Substance Module of the CAS Integration API.  Please contact CAS Custom Services for more information about licensing.  The __[Swagger page](https://helium.cas.org/integration/v1/api-docs/)__ for multiple endpoints is also available as reference.

We will be accessing [CAS Registry®](https://www.cas.org/cas-data/cas-registry), [CAS Reference Collection](https://www.cas.org/cas-data/cas-references), and [CAS Commercial Sources™](https://www.cas.org/cas-data/cas-commercial-sources)

__[Terms of service](https://www.cas.org/legal)__

### Table of Contents
[Getting Started](#Getting_Started)

[Substance Search by CAS Registry Number](#Substance_Search_by_CAS_Registry_Number)
- [Construct Query](#Construct_Query)
- [Display Molecule and Name](#Display)
- [Display Reference Count for Substance](#Display_References)
- [Display Titles](#Display_Titles)

[Commercial Sources by CAS Registry Number](#Commercial_Sources_by_CAS_Registry_Number)

[Substance Search by Structure](#Structure_Search)
- [Construct Query](#Construct_Query)
- [Handling Multiple Pages](#Multipage)
- [Property Data](#Properties)
- [Spectral Data](#Spectral_Data)

[Appendix](#Appendix)
- [Substance Request](#Substance_Request)
- [Search Response](#Search_Response)
- [Substance Summary](#Substance_Summary)
- [Substance Detail](#Substance_Detail)
- [Structure Detail](#Structure_Detail)
- [Available Facets](#Substance_Facets)








<a id="Getting_Started"></a>
## Getting Started
<a id="Authentication"></a>
### Authentication
__To begin utilizing the CAS API, you will first need valid credentials.__  The CAS API handles authorization via __[OAuth2.0](https://oauth.net/2/)__.  The easiest way to authenticate is with a _client ID_ and _client secret_ which can be provided by CAS Custom Services.

To make any request of the API you will need an access token. The access token represents the authorization to access specific parts of CAS' data.  There are several modules available to CAS Integration API users.  Depending on your specific setup, you will have access to all or some of the following capabilities.  For question about your modules, contact CAS Custom Services.
<a id="Import_Libraries"></a>
### Import libraries

In [ ]:
import http.client
import requests
from pprint import pprint
import json
import urllib
from IPython.display import SVG, display, Image


<a id="Set_Credentials"></a>
### Set your supplied credentials from CAS

You will need to enter your companies' client id and secret provided by CAS Custom Services.

In [ ]:
# Input client ID and client secret credentials supplied to you by CAS

clientID = input("Please input your client ID.")
secret = input("Please input your client secret.")

<a id="Authorization_Method"></a>
### Authorization Method

_The recommended method requires a user to access their _client ID_ and  _client secret_  as variables in the code they wish to run the workflow. The workflow requires programmatic submission of the users credentials to obtain an Access Token. This can be used for the duration of the execution of the workflow._

_If the workflow is expected to run for a duration of longer than 24 hours, the user must add code that will periodically update the Access token before it expires. Failure to do this may result in some or all of the requests after the 24-hour period returning a 401 error._

**Definitions**
> **Client ID** is an Organizational-level ID.  Used to initially validate the Organization of a User of the CAS API.  This ID will be provided by a member of CAS Custom Services upon start of the contract.
>
> **Client Secret** is an Organizational-level password/secret.  Used to initially authenticate the Organization of User of the CAS API.  This ID will be provided by a member of CAS Custom Services upon start of the contract.
>
> **Access Token** (_Also known as the Bearer Token_) is a daily token allowing access to the CAS API.  The Access token is passed with a POST or GET request to allow a User to authenticate requests and retrieve data.  This token is valid for _**24 hours**_ from its creation or until a new Access Token is generated via a new request.


<a id="Request_Access_Token"></a>
**Request your Access Token** 

In [ ]:
# specify the content type in the header
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

# insert credentials specified above into payload sent OAuth
payload = ('grant_type=client_credentials&client_id=' + clientID + '&scope=cas_content%20&client_secret=' + secret)

# create a connection
conn = http.client.HTTPSConnection("sso.cas.org")

# make our request
conn.request("POST", "/as/token.oauth2", payload, headers)

# retrieve the response and read it
res = conn.getresponse()
data = res.read()

# read the json and store in a dictionary
response = data.decode("utf-8")
response_json = json.loads(response)

# print POST Response (JSON)
pprint(response_json)
print()

# hold onto this token, we'll need it for future requests to Integration API
access_token = response_json['access_token']

<a id='Substance_Search_by_CAS_Registry_Number'></a>
## Use Case 1: Substance Search by CAS Registry Number®

CAS Registry® is the authoritative source for accurate chemical names, structures, and CAS Registry Numbers®. The CAS Registry Number® or CAS RN is a unique numerical identifier assigned by CAS to every chemical substance described in the open scientific literature.  Scientists, manufacturers, regulators, and data scientists worldwide depend on accurate information related to chemical substances.

Need to determine if a CAS RN is valid and/or validate a substance name and structure.  To retrieve this information, customers can look up the target CAS RN and obtain the necessary information.

<a id='Construct_Query'></a>
### Construct Query

In [ ]:
# make sure our access token is present on requests and that we will receive JSON in return
headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {0}'.format(access_token)}
base_url = 'https://helium.cas.org/integration/v1'

# using the substance module endpoint
module = 'substances'

# Disodium adipate
target_rn = '7486-38-6'

# construct our request
request_url = f'{base_url}/{module}?q={target_rn}'

# make request
resp = requests.get(request_url, headers=headers)

# reusing the response_json variable
response_json = resp.json()

pprint(response_json, depth=5)

<a id="Display"></a>
### Display the Name and Structure Image
Let's display the first substance returned (for a Registry number there will always be a count of one or zero)

We will need to retrieve the `structure` representation from the json.  A `structure` comprises one of more `components`.  Each can contain an `svg` which is displayable.

_Note: not all `components` will have an `svg` available_

In [ ]:
# retrieve the first substance
substances_json = response_json['substances'][0]

# print its preferred name
print(substances_json['name'])
print()

# extract the structure
structure_json = substances_json['structure']

# try to display each component
for component in structure_json['component']:
    comp_svg = component['svg']
    if comp_svg is not None:
        display(SVG(comp_svg))
    else: print("No image available")

<a id="Display_References"></a>
### Display Reference Count For Substance

The patent count is returned with the [Substance Search](#Substance_Search_by_CAS_Registry_Number), but perhaps you might want to know how many total documents from the [CAS Reference Collection](https://www.cas.org/cas-data/cas-references) contain the substance.  In additon, you may also want to pull the titles of those documents to see if they are of interest.

In [ ]:
# obtain substance uri from the above response
sub_uri = substances_json['uri']

#encode the sub_uri
sub_uri = urllib.parse.quote(sub_uri, safe='')

# request count of substances associated with this document but no answers (length=0)
# create request url (this is a different endpoint)
document = f'{base_url}/{module}/{sub_uri}/documents?'

# make request (remember we are reusing our headers with our access token from above)
resp = requests.get(document, headers=headers)

# get the json from the response
document=resp.json()

print()
print('CAS RN:' + str(substances_json['casRn']))
print('NUMBER OF PATENTS:' + str(substances_json['patentCount']))
print('NUMBER OF TOTAL DOCUMENTS:' + str(document['count']))



<a id="Display_Titles"></a>
### Display Titles

For the response above which contains the json containing the documents associated with CAS RN: 7486-38-6, let's take a look at the format of the response and then display the first 10 titles.

In [ ]:
import itertools as mt

# obtain documents json
json = document['documents']

# print first 2 records in json response
pprint(json[0:2])


Now let's look at the titles for the first 10 documents in the response.

In [ ]:
# parse out first 10 titles
for title in json[:10]:
    print ('Title:' +  title['title'])

<a id='Commercial_Sources_by_CAS_Registry_Number'></a>
## Commercial Sources by CAS Registry Number®
CAS Commercial Sources provides an aggregated source for current global chemical catalog data that enables researchers to efficiently identify and compare sourcing options.  Product details including chemical and trade names, CAS Registry Number®, chemical structure, purity, quantity, price, and contact information.

The supplier count is returned with the [Substance Search](#Substance_Search_by_CAS_Registry_Number) if you have access to this endpoint.



In [ ]:
#Using the previously pulled substance_json

print('CAS RN:' + str(substances_json['casRn']))
print('NUMBER OF SUPPLIERS:' + str(substances_json['suppliersCount']))

In the previous substance_json, we obtained the sub_uri.  We can use this uri to obtain information about the suppliers for this substance.  For brevity, a truncated response is provided to only show the first 2 records.
<a id='Construct_Query'></a>
### Construct Query

In [ ]:
#make sure our access token is present on requests and that we will receive JSON in return
headers = {'Content-Type': 'application/json',
           'Authorization': 'Bearer {0}'.format(access_token)}
base_url = 'https://helium.cas.org/integration/v1'

# using the substance module endpoint
module = 'substances'

# obtain substance uri from the previous response
sub_uri = substances_json['uri']

#encode the sub_uri
sub_uri = urllib.parse.quote(sub_uri, safe='')

# construct our request
request_url = f'{base_url}/{module}/{sub_uri}/commercialsources?'

# make request
resp = requests.get(request_url, headers=headers)

# reusing the response_json variable
response_json = resp.json()
truncated_resp = response_json['commercialSources'][:2]

pprint(truncated_resp, depth=3)


We are now able to parse the json to obtain the list of the first 10 suppliers or any other available supplier information for CAS RN 7486-38-6.

In [ ]:
for catalog in response_json['commercialSources'][:10]:
      print(catalog['catalogName'] + ": Ships Within " + catalog['shipsWithin'])

<a id="Structure_Search"></a>
## Use Case 2: Substance Search by Structure

Need to identify all substances with a particular substructure which are commercially available, have a MW between 200 and 400 and do not contain isotopes.  We will compile all the candidates with their name, canonical SMILES, Molecular Formula, the supplier count, and the substance's uri (_will discuss more later_).

To accomplish this search we will utilize facets/filters to select our target candidates.  Many substance searches will return many results and pagination is required (_max is 100 per page_).  We will simulate handling paginated results by requesting 10 results at a time with the `length` parameter.

<a id="Construct_Query"></a>
### Construct Query
We'll define a SMILES string for our search and set the search type to 'substructure'.  Next we will limit our results by specifying that we would like to only return substances which are commercially available, have a MW between 200 and 400, and do not contain isotopes.

<div class="alert alert-block alert-success">
<b>Tip:</b> All query values must be URL encoded (UTF-8) with respect to the values in the queries
</div>

In [ ]:
# our target structure
target_smiles = 'O=C(C=1C=NC=C(C1)B(O)O)C'

#specify a substructure search
structure_search_type = 'sub'

# create our list of facets
facetArray = ["Molecular Weight:200-400",\
             "Isotopes:Not Containing Isotopes",\
              "Commercial Availability:Available"]

<a id="Multipage"></a>
### Handling multipage requests

In [ ]:
# let's keep track of the offset
offset = 0

# define page size
page_size = 2

# indicator that there are more pages
more_pages = True

#keep track of pages already requested
page_count = 1

# let's save off all our substance jsons in a dictionary
# the keys will be CAS Registry Numbers since they are unique
substance_results = {}

# we'll make all our requests to get all the results
while more_pages:
    request_url = f'{base_url}/{module}?str={target_smiles}&strMode={structure_search_type}&offset={offset}&length={page_size}&facet=true'
    # add the facets
    for facet in facetArray:
        request_url += f'&fq={urllib.parse.quote(facet)}'

    # make request (remember we are reusing our headers with our access token from above)
    resp = requests.get(request_url, headers=headers)

    # reusing the response_json variable
    response_json = resp.json()

    # retrieve the number of results
    result_count = response_json['count']

    #Do we need more pages?
    if result_count/page_size <= page_count:
        print(f'There are {result_count} results')
        more_pages = False
    else:
        page_count += 1

    # increment to get the next page
    offset += page_size

    # add this page's results
    for substance in response_json['substances']:
        substance_results[substance['casRn']] = [substance['name'],substance['canonicalSmiles'], substance['suppliersCount'], substance['uri'], substance['molecularFormula']]

# let's print them pretty
pprint(substance_results)

<a id="Properties"></a>
### Display Preferred Properties for a Substance
In order to retrieve a full Registry record, you will need to find the uri for the target substance.  This can be found by querying by the CAS Registry Number using the above endpoint and retrieving the uri from the record.  However, generally speaking the uri can be created in the form `substance/pt/[no_dash_RN]` where no_dash_RN is the CAS Registry Number with the '-' removed.
_for example 1256913-43-5 would become 'substance/pt/1256913435'_

The detail record contains a few extra parameters including `properties` which we are interested in for this case.  See the full [JSON description](TODO) of the detail in appendix

In [ ]:
# let's choose the last result from above
last_key = list(substance_results.keys())[-1]

# retrieve the data from dictionary
substance_data = substance_results[last_key]

# we must encode the '/' in the uri or we will get an error
target_uri = urllib.parse.quote(substance_data[3], safe='')

# construct our request
request_url = f'{base_url}/{module}/{target_uri}'

# make request (remember we are reusing our headers with our access token from above)
resp = requests.get(request_url, headers=headers)

# reusing the response_json variable
response_json = resp.json()

# extract the properties
exp_properties = response_json['substance']['properties']

#print the RN and its properties
print(last_key)
pprint(exp_properties, depth=3)

<a id="Spectral_Data"></a>
### Display Experimental Spectra for a Substance



In [ ]:
# let's choose a different substance which has experimental IR Spectra
cas_rn = '71-43-2'
substance_uri = 'substance/pt/71432'

# we must encode the '/' in the uri or we will get an error
target_uri = urllib.parse.quote(substance_uri, safe='')

# construct our request
request_url = f'{base_url}/{module}/{target_uri}'

# make request (remember we are reusing our headers with our access token from above)
resp = requests.get(request_url, headers=headers)

# reusing the response_json variable
response_json = resp.json()

# extract the spectra information
exp_spectra = response_json['substance']['capturedExperimentalSpectra']['experimentalIRSpectra'][0]
#print the RN and its spectra information
print(cas_rn)
pprint(exp_spectra, depth=3)

From the response above, we can leverage the 'id' of the spectrum plus the substance uri to obtain the png image of the spectrum.

In [ ]:
# obtain the spectra id and attribution
exp_spectra_attribution = exp_spectra['dataSource']['attributionGroup']['fullAttributionText']
exp_spectra_id = exp_spectra['id']
exp_spectra_spectrometer_condition = exp_spectra['experimentalSpectrumSpectrometerCondition']['displayValue']

# we must encode the '/' in the uri or we will get an error
target_uri = urllib.parse.quote(substance_uri, safe='')

# construct our request
request_url = f'{base_url}/{module}/{target_uri}/captured/experimental/spectra/{exp_spectra_id}/image'

# change headers for image
headers = {'Content-Type': 'image/png', 'cache-control': 'no-cache',
           'Authorization': 'Bearer {0}'.format(access_token)}
#make request (new headers)
resp = requests.get(request_url, headers=headers)

# print and display substance and spectra information
print("CAS Registry Number: " + cas_rn)
print("IR Spectra ID: " + exp_spectra_id)
print(exp_spectra_attribution)
print()
display(Image(resp.content))
print(exp_spectra_spectrometer_condition)





<a id="Appendix"></a>
# Appendix
<a id="Substance_Request"></a>
## Substance Request

A request for substance information can be defined using the endpoint, `https://helium.cas.org/integration/v1/substances`, with the following parameters:

>**q** is a *query string* used for a CAS Registry Number or a substance name search
> CAS Registry Numbers must contain the proper format *(regex \d+-\d\d-\d)*
> *This parameter can only be used if the `str` parameter is absent*

>**str** - is a *substance structure query*
> This query can be a molfile, SMILES, InChI, or CXF JSON (exported by structure editor in SciFinder or STN). Maximum length is 20000.
> *Note that this must be a URL encoded string.*
> *This parameter can only be used if the `q` parameter is absent*

>**strMode** is the *type of structure search* to be conducted.
> There are three options: `drw` (As Drawn), `sub` (Substructure), or `sim` (Similarity).
>This parameter can only be used if the `str` parameter is defined.
> *Default: `drw`*

>**offset** is the *offset* from the start of the result set to be returned in the response.
> This offset indicates which ordinal to start returning results.  Along with the `length` parameter a "page" of answers can be defined.
> _Default: 0, minimum: 0_

>**length** is the *maximum number of results* to be returned in the response.
> This length indicates the maximum number of results to return starting at the 'offset' parameter.
> _Default: 100, minimum: 0, maximum: 100_

>**echo** is an indication of whether the response should contain the request parameters in the response.
> When set to true the response will include the parameters explicitly specified by the request.
> _Default: false_

>**strHighlight** is an indication of whether *structure highlighting* should be shown in the structure images (svg) returned
> When set to `true`, the structure images (svg) returned will be highlighted.
> _This parameter can only be used if the `str` parameter is defined_
> _Default is false_

>**facet** is an indication of whether to return the *facets* in the response.
> Facets summarize the results by slicing into facet bins.  Each facet bin will indicate the number of results in that bin.
> When set to true, the response will contain a list of faceted bins.
> _Default is false_

>**fq** is an array of *facet queries*.
 > A facet query that can be used to filter results. Multiple filters can be combined (AND'ed) to make the constraints more restrictive (including more filters typically returns less results). This example shows a) Number of Components:1, b) Isotopes:Not Containing Isotopes, and c) Commercial Availability:Available.

### Example
```
https://helium.cas.org/integration/v1/substances?str=O%3DC%28C%3D1C%3DNC%3DC%28C1%29B%28O%29O%29C \
    &strMode=sub \
    &offset=0 \
    &length=10 \
    &fq=Number%20of%20Components%3A1 \
    &fq=Isotopes%3ANot%20Containing%20Isotopes \
    &fq=Commercial%20Availability%3AAvailable
```
<div class="alert alert-block alert-success">
<b>Tip:</b> If input contains JSON reserved characters, they should be escaped properly:
</div>

```
 - Backspace is replaced with \b
 - Form feed is replaced with \f
 - Newline is replaced with \n
 - Carriage return is replaced with \r
 - Tab is replaced with \t
 - Double quote is replaced with \"
 - Backslash is replaced with \\
```


<a id="Search_Response"></a>
## Substance Search Response JSON
Results of a substance search.  The response is in [JSON](https://en.wikipedia.org/wiki/JSON#Syntax) format

The Search Response utilizes an additional important construct, a [Substance Summary](#SubstanceSummary).

>**count** is the *number of results* returned from the query.
> *This count can be 0 or more*
> *Note: This count is not the number of `substances` being returned in this response unless the count is less than the length attribute.*
> **Example:** A structure search yields a count of 86; however, with length=3, only the first 3 *substance summaries* will be in the Substance Search Response.

>**facet** an array of *facets* for the result set.
> Facets contain facet bins which are subdivided into small slices based on the values. Facets summarize the results by slicing into facet bins.  Each facet bin will indicate the number of results in that bin.
> *Note: This is only present if the request has the `facet` parameter set to `true`*

>**request** an array of *request parameters* from the original request.
> The request parameters are defined above in the [Substance Request](#Substance_Request) Appendix
> *Note: This is only present if the request has the `echo` parameter set to `true`*

>**substances** an array of *substance summaries* returned in the response.
> The number of `substances` is determined by the `length` parameter on the request or the count, whichever is less

### Example
```
{'count': 1,
 'facets': [...],      # optional
 'request': {...},     # optional
 'substances': [...]
 }
```

<a id="Substance_Summary"></a>
## Substance Summary JSON
<a id="SubstanceSummary"></a>This is the definition of a substance summary of Registry entries.  A Substance Summary contains many of the fields available for this substance but does not include any experimental property data.

The Substance Summary utilizes an additional important construct, a `structure`.  The `structure` is described in the - [Structure Detail JSON](#Structure_Detail) below.
> **canonicalSmiles** is the *Canonical SMILES* format for this substance.
> Canonical SMILES follows the same general encoding method as SMILES but is a unique representation of a substance. This is achieved by standardizing the encoding starting point.  CAS Canonical SMILES do not contain isotopic or stereochemistry of a molecule.

> **casRn** is the preferred *CAS Registry Number* for this substance.
> The CAS RN is a numeric identifier that can contain up to 10 digits, divided by hyphens into three parts. The right-most digit is a check digit used to verify the validity and uniqueness of the entire number.

> **componentRN** is the *CAS Registry Number* of the component being represented.
> Please ignore in the context of the Substance Summary.  See - [Structure Detail JSON](#Structure_Detail) below.

> **deletedRNs** is an array of *CAS Registry Numbers* which had previously been assigned to a substance but later changed to another Registry Number.
> If there are no deleted *CAS Registry Numbers* this will field will contain an empty array.

> **hitName** is the *name* of the substance which was matched against by the query.
> If the query contained no substance name this field will contain the value "null".

> **hitRN** is the *CAS Registry Number* of the substance which was matched against by the query.
> The hit Registry Number will be surrounded by the tag `<mark>`.  If the query contained no registry number this field will contain the value "null".

> **inchi** is the *The International Chemical Identifier* for this substance.
> This representation is a textual identifier for chemical substances developed by the InChI Trust.

> **isomericSmiles** is the *Isomeric SMILES* format for this substance.
> Isomeric SMILES follows the same general encoding method as SMILES but allows for specifying isotopism and stereochemistry of a molecule. Information on isotopism is indicated by the integral atomic mass preceding the atomic symbol.

> **molecularFormula** is the *molecular formula* for the substance
> The *molecular formula* of a substance is a formula which giving the number of atoms of each of the elements present in one molecule of a specific compound.  The CAS molecular formula is represented in "Hill Order" which lists the number of carbon atoms first, then the number of hydrogen atoms, and finally all other elements in alphabetical order.  If a compound doesn't contain carbon, then all elements are listed alphabetically, including hydrogen.

> **molfiles** is a *format for representing the structure of a substance* developed by Molecular Design Limited (MDL).
> A molfile contains information about the atoms, bonds, connectivity and coordinates of a molecule. The molfile consists of some header information, the Connection Table (CT) containing atom info, then bond connections and types, followed by sections for more complex information.

> **name** is the *CAS Display Name* for this substance.
> There is only one name associated with the CAS Display Name (a name used for product display). 

> **patentCount** is the *number of patents* in the CAS Content Collection which contain this substance
> *This count can be 0 or more*

> **preferredRn** is the *CAS Registry Number* of this substance which is preferred to be used to represent this substance.
> This is usually the same value as the casRN unless the query contained an RN which matched on a deleted CAS Registry Number or an Alternate CAS Registry Number.  If an Alternate RN is used in the query, the summary response will bring back the RN entered as well as the preferred RN.

> **smiles** is the SMILES representation of this substance.
> Regular SMILES are a linear representation of a chemical solely based on its molecular graph. This representation does not specify where on the structure to begin encoding nor does it encode chiral or isotopic information. Consequently, SMILES are not unique representations

> **structure** is a construct to describe the substance with *structural characteristics*
> This is described below in the - [Structure Detail JSON](#Structure_Detail)

> **supplierCount** is the *number of suppliers* in the CAS Content Collection who offer this substance for sale 
> *This count can be 0 or more*

> **structureMatchPercentage** is the *percentage match* of the substance to a similarity structure query.
 > _Note: only returned when `strMode` is set to `sim`_.

> **uri** is a unique key specific to the Integration API for this substance.
>This uri can be used to retrieve a detail record for the substance, spectra, suppliers (commercial sources), or the documents in which this substance appears.



<a id="Substance_Detail"></a>
## Substance Detail JSON
This is the definition of a substance for Registry entries.  The Substance construct is very similar to a Substance Summary above.  The definitions for identical parameters will not be described here.

The Substance utilizes an additional important construct, a `structure`.  The `structure` is described in the [Structure JSON](Structure_Detail) below.
>**entryDate** is the *date* the substance was first introduced to the CAS Registry.

>**properties** is an array of _experimental properties_

>**regulatoryDataAvailable** is a boolean value (`True` | `False`) indicating whether this substance is regulated in a jurisdiction.


### Example
```
{'alternateRns': [],
 'canonicalSmiles': 'O=C(C=1C=NC=C(C1)B(O)O)C',
 'capturedExperimentalProperties': {...},
 'capturedExperimentalSpectra': {...},
 'casRn': '1033745-21-9',
 'componentRn': None,
 'deletedRns': [],
 **'entryDate': '2008-07-11',**
 'hitName': None,
 'hitRn': None,
 'inchi': 'InChI=1S/C7H8BNO3/c1-5(10)6-2-7(8(11)12)4-9-3-6/h2-4,11-12H,1H3',
 'isomericSmiles': 'B(O)(O)C=1C=C(C(C)=O)C=NC1',
 'molecularFormula': 'C<sub>7</sub>H<sub>8</sub>BNO<sub>3</sub>',
 'molecularWeight': 164.95,
 'molfiles': [...],
 'name': '<em>B</em>-(5-Acetyl-3-pyridinyl)boronic acid',
 'patentCount': 7,
 'preferredRn': '1033745-21-9',
 **'properties': {...},**
 **'regulatoryDataAvailable': False,**
 'smiles': 'O=C(C=1C=NC=C(C1)B(O)O)C',
 'structure': {...},
 'suppliersCount': '50',
 **'synonyms': [...],**
 'uri': 'substance/pt/1033745219'}}
 ```



<a id="Structure_Detail"></a>
## Structure Detail JSON

A Structure Detail contains information regarding the chemical structural representation of a Registry entry. If a substance has multiple components, there will be multiple component blocks within the JSON.  The definitions for identical parameters will not be described here. 


>**index**: is the *date* the substance was first introduced to the CAS Registry.

>**coefficient**: coefficient of the component in a given multi-component substance

>**singleAtomFragment**:

>**highlighted**: is a boolean value (`True` | `False`) indicating whether this substance returns a highlighted svg

>**svg**: is a scalable vector graphic of a substance. For multi-component substances, there will be an svg for each component.

>**stereoMessages**: is a field which contains pertinent sterochemical information such as "Absolute stereochemistry shown".

>**diagnosticMessages**: is a field which indicates a problem with the query.


### Example
```
"structure": {
      "component": [
        {
          "uri": "substance/pt/1033745219",
          "casRn": "1033745-21-9",
          "index": null,
          "coefficient": null,
          "singleAtomFragment": false,
          "highlighted": false,
          "svg": "...",
          "stereoMessages": [],
          "diagnosticMessages": [],
          "structures": []
        }
      ]
    }
```

<a id="Substance_Facets"></a>
## Available Facets

<div class="alert alert-block alert-success">
<b>Tip:</b> Facets need to be URL encoded.
</div>

>Use of a facet within a query is constructed as follows:
>>Facet Name:Value
>>Example: facet=true&fq=Reaction Role:Product (Encoded: facet=true&fq=Reaction%20Role%3AProduct)

#### Reaction Role
>Product
>
>Reactant
>
>Reagent
>
>Catalyst
>
>Solvent


#### Number of Components
>1
> 
>2
> 
>3
> 
>4
> 
>5 or more
> 

#### Commercial Availability
>Available
> 
>Not Available
> 

#### Substance Class
>Alloy
> 
>Coordination Compound
> 
>Element
> 
>General Derivative
> 
>Generic Registration
> 
>Incompletely Defined Substance
> 
>Manual Registration
> 
>Mineral
> 
>Mixture
> 
>Nucleic Acid Sequence
> 
>Organic/Inorganic Small Molecule
> 
>Polymer
> 
>Protein/Peptide Sequence
> 
>Radical Ion
> 
>Registered Concept
> 
>Ring Parent
> 
>Salt And Compound With
> 
>Subatomic Particle
> 
>Tabular Inorganic
> 

#### Isotopes
>Not Containing Isotope
> 
>Containing Isotopes
> 

#### Molecular Weight
>164
> 
>300-400
> 

#### Stereochemistry Input Has Stereo
>Absolute Stereo Match
> 
>Absolute Stereo Mirror Image
> 
>No Stereo in Answer Structure
> 
>Relative Stereo Match
> 
>Stereo that Doesn't Match Query
> 
>Double Bond Geometry As Drawn
> 

#### Stereochemistry Input Has No Stereo
>Stereo in Answer Structure
> 
>No Stereo in Answer Structure
> 

#### Similarity Score
*Only used when strMode=sim*
> 
>\>=99
> 
>95-98
> 
>90-94
> 
>85-89
> 
>80-84
> 
>75-79
> 
>70-74
> 
>65-69
> 
>60-64
> 

#### Substance Patents
>Has Patents
> 
>Has No Patents
> 

#### Bioactivity Indicator
> A bioactivity indicator is a predefined term used to describe the bioactivity of a substance.  This is a collection of terms that can be used to identify relationships between substances and documents.

 <details>
 <summary>List of available bioactivity indicators</summary>
 
>5-HT1 agonists
> 
 >5-HT1 antagonists
> 
 >5-HT1A agonists
> 
 >5-HT1A antagonists
> 
 >5-HT1B agonists
> 
 >5-HT1B antagonists
> 
 >5-HT1D agonists
> 
 >5-HT1D antagonists
> 
 >5-HT1F agonists
> 
 >5-HT2 agonists
> 
 >5-HT2 antagonists
> 
 >5-HT2A agonists
> 
 >5-HT2A antagonists
> 
 >5-HT2B agonists
> 
 >5-HT2B antagonists
> 
 >5-HT2C agonists
> 
 >5-HT2C antagonists
> 
 >5-HT3 agonists
> 
 >5-HT3 antagonists
> 
 >5-HT4 agonists
> 
 >5-HT4 antagonists
> 
 >5-HT5 agonists
> 
 >5-HT5 antagonists
> 
 >5-HT6 agonists
> 
 >5-HT6 antagonists
> 
 >5-HT7 agonists
> 
 >5-HT7 antagonists
> 
 >5-HT agonists
> 
 >5-HT antagonists
> 
 >5-HT reuptake inhibitors
> 
 >5-Lipoxygenase-activating protein inhibitors
> 
 >14α-Demethylase inhibitors
> 
 >AMP-activated protein kinase activators
> 
 >AMPA receptor agonists
> 
 >AMPA receptor antagonists
> 
 >ATPase inhibitors
> 
 >Abortifacients
> 
 >Acetaldehyde dehydrogenase inhibitors
> 
 >Acetolactate synthase inhibitors
> 
 >Acetyl-coenzyme A carboxylase inhibitors
> 
 >Acetylcholine release inhibitors
> 
 >Addison disease medications
> 
 >Adenosine A1 receptor agonists
> 
 >Adenosine A1 receptor antagonists
> 
 >Adenosine A2 receptor agonists
> 
 >Adenosine A2 receptor antagonists
> 
 >Adenosine A3 receptor agonists
> 
 >Adenosine A3 receptor antagonists
> 
 >Adenosine deaminase inhibitors
> 
 >Adenosine receptor agonists
> 
 >Adenosine receptor antagonists
> 
 >Adenovirus-based vaccines
> 
 >Adoptive immunotherapy
> 
 >Adrenergic reuptake inhibitors
> 
 >Adrenoceptor agonists
> 
 >Adrenoceptor antagonists
> 
 >Advanced glycation end product inhibitors
> 
 >Alcohol dehydrogenase inhibitors
> 
 >Alcohol deterrents
> 
 >Aldose reductase inhibitors
> 
 >Aldosterone synthase inhibitors
> 
 >Alkylating agents
> 
 >Allergy inhibitors
> 
 >Allergy vaccines
> 
 >Allosteric modulators
> 
 >Alzheimer vaccines
> 
 >Amebicides
> 
 >Aminoacyl-tRNA synthetase inhibitors
> 
 >Aminoglycoside antibiotics
> 
 >Amyloid formation inhibitors
> 
 >Amyloid β-peptide-binding alcohol dehydrogenase inhibitors
> 
 >Anabolic agents
> 
 >Analgesics
> 
 >Analgesics, central analgesics
> 
 >Androgen receptor agonists
> 
 >Androgen receptor antagonists
> 
 >Anesthetics
> 
 >Anesthetics, i.v. injections
> 
 >Angiotensin II receptor antagonists
> 
 >Angiotensin II receptor type 1 antagonists
> 
 >Angiotensin II receptor type 2 antagonists
> 
 >Angiotensin-converting enzyme inhibitors
> 
 >Ansamycin antibiotics
> 
 >Antacids
> 
 >Anthelmintics
> 
 >Anthracycline antibiotics
> 
 >Anthrax vaccines
> 
 >Anti-Alzheimer agents
> 
 >Anti-HIV agents
> 
 >Anti-Huntington agents
> 
 >Anti-acne agents
> 
 >Anti-adenovirus vaccines
> 
 >Anti-aging agents
> 
 >Anti-apoptotic agents
> 
 >Anti-cellulite agents
> 
 >Anti-hepatitis B virus agents
> 
 >Anti-hepatitis C virus agents
> 
 >Anti-hypoxic agents
> 
 >Anti-infective agents
> 
 >Anti-inflammatory agents
> 
 >Anti-inflammatory analgesics
> 
 >Anti-ischemic agents
> 
 >Anti-lipopolysaccharide factors
> 
 >Anti-multiple sclerosis agents
> 
 >Anti-neurodegenerative agents
> 
 >Anti-protozoan vaccines
> 
 >Anti-restenotic agents
> 
 >Antiandrogens
> 
 >Antianemic agents
> 
 >Antianginal agents
> 
 >Antiangiogenic agents
> 
 >Antiarrhythmics
> 
 >Antiarrhythmics, antifibrillatory
> 
 >Antiarrhythmics, bradycardiacs
> 
 >Antiarteriosclerotics
> 
 >Antiarthritics
> 
 >Antiasthmatics
> 
 >Antiatherosclerotics
> 
 >Antibacterial agents
> 
 >Antibacterial agents, biostatic agents
> 
 >Antibacterial agents, sustained-release
> 
 >Antibacterial vaccines
> 
 >Antibiotics
> 
 >Antibody mimetics
> 
 >Anticholelithic agents
> 
 >Anticholesteremic agents
> 
 >Anticoagulant antidotes
> 
 >Anticoagulants
> 
 >Anticoagulants, oral
> 
 >Anticonvulsants
> 
 >Antidandruff hair preparations
> 
 >Antidepressants
> 
 >Antidiabetic agents
> 
 >Antidiarrheals
> 
 >Antidiuretics
> 
 >Antidiuretics, antinatriuretics
> 
 >Antidotes
> 
 >Antiemetics
> 
 >Antiestrogens
> 
 >Antifibrinolytic agents
> 
 >Antifibrotic agents
> 
 >Antiflatulents
> 
 >Antifolates
> 
 >Antiglaucoma agents
> 
 >Antigout agents
> 
 >Antihistamines
> 
 >Antihypertensives
> 
 >Antihypotensives
> 
 >Antimalarials
> 
 >Antimicrobial agents
> 
 >Antimicrobial peptides
> 
 >Antimigraine agents
> 
 >Antimitotic agents
> 
 >Antimycobacterial agents
> 
 >Antineoplastons
> 
 >Antiobesity agents
> 
 >Antiosteoporotic agents
> 
 >Antiparkinsonian agents
> 
 >Antiprogestins
> 
 >Antiproliferative agents
> 
 >Antipruritic agents
> 
 >Antipsoriatic agents
> 
 >Antipsychotic antidepressants
> 
 >Antipsychotics
> 
 >Antipyretics
> 
 >Antirheumatic agents
> 
 >Antispasmodics
> 
 >Antithrombotics
> 
 >Antithyroid agents
> 
 >Antitumor agents
> 
 >Antitumor antibiotics
> 
 >Antitussives
> 
 >Antiulcer agents
> 
 >Antivenoms
> 
 >Antiviral agents
> 
 >Antiviral vaccines
> 
 >Anxiolytics
> 
 >Aphrodisiacs
> 
 >Appetite depressants
> 
 >Appetite stimulants
> 
 >Arachidonate lipoxygenase inhibitors
> 
 >Aromatase inhibitors
> 
 >Aromatic amino acid decarboxylase inhibitors
> 
 >Aspartic protease inhibitors
> 
 >Astringents
> 
 >Aurora kinase inhibitors
> 
 >Autoimmune disease medications
> 
 >Autophagy inhibitors
> 
 >Autotaxin inhibitors
> 
 >Autovaccines
> 
 >Azalide antibiotics
> 
 >Bacterial meningitis vaccines
> 
 >Basic fibroblast growth factor inhibitors
> 
 >Benzodiazepine receptor agonists
> 
 >Benzodiazepine receptor antagonists
> 
 >Bile acid sequestrants
> 
 >Biological response modifiers
> 
 >Biopharmaceuticals
> 
 >Blood coagulation factor inhibitors
> 
 >Bone resorption inhibitors
> 
 >Boophilus vaccines
> 
 >Bradykinin B1 receptor antagonists
> 
 >Bradykinin B2 receptor antagonists
> 
 >Bradykinin antagonists
> 
 >Bromodomain and extra-terminal domain-containing protein inhibitors
> 
 >Bronchodilators
> 
 >Bruton tyrosine kinase inhibitors
> 
 >CAR T-cell immunotherapy
> 
 >CCR1 chemokine receptor antagonists
> 
 >CCR2 chemokine receptor antagonists
> 
 >CCR3 chemokine receptor antagonists
> 
 >CCR4 chemokine receptor antagonists
> 
 >CCR5 chemokine receptor antagonists
> 
 >CCR7 chemokine receptor antagonists
> 
 >COPD medications
> 
 >CXCR3 chemokine receptor antagonists
> 
 >CXCR4 chemokine receptor antagonists
> 
 >Calcimimetics
> 
 >Calcineurin inhibitors
> 
 >Calcium channel blockers
> 
 >Calcium channel openers
> 
 >Calcium pump inhibitors
> 
 >Calcium-calmodulin-dependent protein kinase inhibitors
> 
 >Cancer prevention agents
> 
 >Cancer vaccines
> 
 >Cannabinoid receptor agonists
> 
 >Cannabinoid receptor antagonists
> 
 >Cannabinoids
> 
 >Carbamoyl phosphate synthetase I activators
> 
 >Carbapenems
> 
 >Carbohydrate mimetics
> 
 >Carbon monoxide-releasing molecules
> 
 >Carbonic anhydrase inhibitors
> 
 >Cardiac glycosides
> 
 >Cardioplegic agents
> 
 >Cardioprotective agents
> 
 >Cardiotonics
> 
 >Cardiovascular agents
> 
 >Caspase inhibitors
> 
 >Catechol O-methyltransferase inhibitors
> 
 >Celiac disease medications
> 
 >Cell differentiation inducers
> 
 >Cell migration inhibitors
> 
 >Cell-based biopharmaceuticals
> 
 >Central nervous system agents
> 
 >Central nervous system stimulants
> 
 >Cephamycin antibiotics
> 
 >Cerebral vasodilators
> 
 >Checkpoint kinase inhibitors
> 
 >Chemokine receptor antagonists
> 
 >Chemosensitizers, pharmaceutical
> 
 >Chickenpox vaccines
> 
 >Chloride channel blockers
> 
 >Chloride channel openers
> 
 >Cholera vaccines
> 
 >Choleretics
> 
 >Cholesterol absorption inhibitors
> 
 >Cholesterol acyltransferase inhibitors
> 
 >Cholinergic agents
> 
 >Cholinergic agonists
> 
 >Cholinergic antagonists
> 
 >Cholinesterase inhibitors
> 
 >Chondroprotective agents
> 
 >Chronotropics
> 
 >Chronotropics, negative
> 
 >Chymotrypsin inhibitors
> 
 >Cicatrizants
> 
 >Class I antiarrhythmics
> 
 >Class II antiarrhythmics
> 
 >Class III antiarrhythmics
> 
 >Class IV antiarrhythmics
> 
 >Class Ia antiarrhythmics
> 
 >Class Ib antiarrhythmics
> 
 >Class Ic antiarrhythmics
> 
 >Cobra venom factors
> 
 >Coccidiostats
> 
 >Cognition enhancers
> 
 >Collagenase inhibitors
> 
 >Combination vaccine DTP-HepB
> 
 >Combination vaccine DTP-HepB-Hib
> 
 >Combination vaccine DTP-IPV-Hib
> 
 >Combination vaccine DTaP-Hib
> 
 >Combination vaccine DTaP-IPV
> 
 >Combination vaccine DTaP-IPV-Hib
> 
 >Combination vaccines
> 
 >Comedolytic agents
> 
 >Complement inhibitors
> 
 >Complement-activating glycoproteins
> 
 >Continuous erythropoietin receptor activators
> 
 >Contraceptive vaccines
> 
 >Contraceptives
> 
 >Contraceptives, postcoital
> 
 >Contraceptives, s.c.
> 
 >Contraceptives, spermicidal
> 
 >Convulsants
> 
 >Corticotropin releasing factor receptor antagonists
> 
 >Cosmeceuticals
> 
 >Crohn disease medications
> 
 >Cyclin-dependent kinase inhibitors
> 
 >Cyclooxygenase 1 inhibitors
> 
 >Cyclooxygenase 2 inhibitors
> 
 >Cyclooxygenase inhibitors
> 
 >Cytidine deaminase inhibitors
> 
 >Cytochrome P450 CYP1A2 inducers
> 
 >Cytochrome P450 CYP1A2 inhibitors
> 
 >Cytochrome P450 CYP2B6 inducers
> 
 >Cytochrome P450 CYP2B6 inhibitors
> 
 >Cytochrome P450 CYP2C8 inducers
> 
 >Cytochrome P450 CYP2C8 inhibitors
> 
 >Cytochrome P450 CYP2C9 inducers
> 
 >Cytochrome P450 CYP2C9 inhibitors
> 
 >Cytochrome P450 CYP2C19 inducers
> 
 >Cytochrome P450 CYP2C19 inhibitors
> 
 >Cytochrome P450 CYP2D6 inducers
> 
 >Cytochrome P450 CYP2D6 inhibitors
> 
 >Cytochrome P450 CYP2E1 inducers
> 
 >Cytochrome P450 CYP2E1 inhibitors
> 
 >Cytochrome P450 CYP3A4 inducers
> 
 >Cytochrome P450 CYP3A4 inhibitors
> 
 >Cytochrome P450 CYP3A inducers
> 
 >Cytochrome P450 CYP3A inhibitors
> 
 >Cytochrome P450 enzyme inducers
> 
 >Cytochrome P450 enzyme inhibitors
> 
 >Cytokine inhibitors
> 
 >Cytomegalovirus vaccines
> 
 >Cytoprotective agents
> 
 >Cytotoxic agents
> 
 >DNA minor groove binders
> 
 >DNA polymerase inhibitors
> 
 >DNA vaccines
> 
 >Decongestants
> 
 >Decongestants, nasal
> 
 >Delayed hypersensitivity inhibitors
> 
 >Demulcents
> 
 >Dendritic cell-based vaccines
> 
 >Dengue vaccines
> 
 >Dermatological agents
> 
 >Dermatological antiulcer agents
> 
 >Dihydroorotate dehydrogenase inhibitors
> 
 >Dihydropyridine calcium channel blockers
> 
 >Dipeptidyl peptidase 4 inhibitors
> 
 >Dipeptidyl peptidase inhibitors
> 
 >Diphtheria vaccines
> 
 >Diphtheria-tetanus vaccines
> 
 >Diphtheria-tetanus-acellular pertussis vaccines
> 
 >Diphtheria-tetanus-pertussis vaccines
> 
 >Direct factor Xa inhibitors
> 
 >Disinfectants
> 
 >Disinfectants, sustained-release
> 
 >Diuretics
> 
 >Diuretics, loop
> 
 >Dopamine D1 agonists
> 
 >Dopamine D1 antagonists
> 
 >Dopamine D2 agonists
> 
 >Dopamine D2 antagonists
> 
 >Dopamine D3 agonists
> 
 >Dopamine D3 antagonists
> 
 >Dopamine D4 agonists
> 
 >Dopamine D4 antagonists
> 
 >Dopamine agonists
> 
 >Dopamine antagonists
> 
 >Dopamine reuptake inhibitors
> 
 >Drug resistance modulators
> 
 >Dual inhibitors
> 
 >Dual vasopeptidase inhibitors
> 
 >Dual-specificity protein kinase WEE1 inhibitors
> 
 >Dynamin inhibitors
> 
 >Ebola vaccines
> 
 >Echinocandins
> 
 >Ectoparasiticides
> 
 >Efflux pump inhibitors
> 
 >Elastase inhibitors
> 
 >Emetics
> 
 >Endoparasiticides
> 
 >Endopeptidase inhibitors
> 
 >Endosomolytic agents
> 
 >Endothelin ETA receptor antagonists
> 
 >Endothelin ETB receptor antagonists
> 
 >Endothelin receptor antagonists
> 
 >Endothelin-converting enzyme 1 inhibitors
> 
 >Enoyl-acyl-carrier-protein reductase InhA inhibitors
> 
 >Enzyme activators
> 
 >Enzyme inhibitors
> 
 >Enzyme mimics
> 
 >Enzyme modulators
> 
 >Epigenetic drugs
> 
 >Epithelial sodium channel agonists
> 
 >Erectile dysfunction agents
> 
 >Erythropoiesis-stimulating agents
> 
 >Erythropoietin receptor agonists
> 
 >Estrogen receptor agonists
> 
 >Exon-skipping antisense oligonucleotides
> 
 >Expectorants
> 
 >Factor IIa inhibitors
> 
 >Factor VIIa inhibitors
> 
 >Factor VIIa-tissue factor inhibitors
> 
 >Farnesoid X receptor agonists
> 
 >Farnesyltransferase inhibitors
> 
 >Fasciolicides
> 
 >Fatty acid synthesis inhibitors
> 
 >Female contraceptives
> 
 >Fibrinolytics
> 
 >Fibroblast growth factor receptor antagonists
> 
 >Filaricides
> 
 >Fumigant disinfectants
> 
 >Fungal vaccines
> 
 >Fungicides
> 
 >G protein inhibitors
> 
 >G protein-coupled receptor 40 agonists
> 
 >G protein-coupled receptor GPR142 agonists
> 
 >GABA agents
> 
 >GABA receptor agonists
> 
 >GABA receptor antagonists
> 
 >GABA reuptake inhibitors
> 
 >GABAA receptor agonists
> 
 >GABAA receptor antagonists
> 
 >GABAB receptor agonists
> 
 >GABAB receptor antagonists
> 
 >GTPase activators
> 
 >Ganglionic blocking agents
> 
 >Gastrointestinal agents
> 
 >Gastrointestinal antiulcer agents
> 
 >Gastrointestinal antiulcer agents, duodenal ulcer
> 
 >Gastrointestinal antiulcer agents, peptic ulcer
> 
 >Gastroprotective agents
> 
 >Gelatinase inhibitors
> 
 >General anesthetics
> 
 >Gentamicins
> 
 >Glucagon receptor agonists
> 
 >Glucagon receptor antagonists
> 
 >Glucagon-like peptide-1 receptor agonists
> 
 >Glucocorticoid antagonists
> 
 >Glucocorticoid receptor agonists
> 
 >Glucocorticoid receptor antagonists
> 
 >Glucokinase activators
> 
 >Glutamate receptor agonists
> 
 >Glutamate receptor antagonists
> 
 >Glutamate receptor modulators
> 
 >Glutamine synthetase inhibitors
> 
 >Glycine receptor agonists
> 
 >Glycine receptor antagonists
> 
 >Glycogen phosphorylase inhibitors
> 
 >Glycogen synthase kinase 3 inhibitors
> 
 >Glycoprotein gp130 antagonists
> 
 >Glycosidase inhibitors
> 
 >GnRH agonists
> 
 >GnRH antagonists
> 
 >Goitrogens
> 
 >Gonadotropin antagonists
> 
 >Graves disease medications
> 
 >Growth hormone secretagogues
> 
 >Growth hormone-releasing hormone antagonists
> 
 >HCV proteinase inhibitors
> 
 >HIF prolyl hydroxylase inhibitors
> 
 >HIV fusion inhibitors
> 
 >HIV protease inhibitors
> 
 >HIV vaccines
> 
 >HMG-CoA reductase inhibitors
> 
 >Haemophilus influenzae type A vaccines
> 
 >Haemophilus influenzae type B vaccines
> 
 >Haemophilus influenzae vaccines
> 
 >Hair growth inhibitors
> 
 >Hair growth stimulants
> 
 >Halichondrins
> 
 >Heat-shock protein 70 inhibitors
> 
 >Heat-shock protein 90 inhibitors
> 
 >Heat-shock protein inhibitors
> 
 >Helicase inhibitors
> 
 >Hematologic agents
> 
 >Hemostatics
> 
 >Heparin antagonists
> 
 >Hepatitis A vaccines
> 
 >Hepatitis B vaccines
> 
 >Hepatitis C vaccines
> 
 >Hepatoprotective agents
> 
 >Herpesvirus vaccines
> 
 >Highly active antiretroviral therapy agents
> 
 >Histamine H1 receptor agonists
> 
 >Histamine H1 receptor antagonists
> 
 >Histamine H2 receptor agonists
> 
 >Histamine H2 receptor antagonists
> 
 >Histamine H3 receptor agonists
> 
 >Histamine H3 receptor antagonists
> 
 >Histamine H4 receptor agonists
> 
 >Histamine H4 receptor antagonists
> 
 >Histamine receptor agonists
> 
 >Histone acetyltransferase inhibitors
> 
 >Histone deacetylase inhibitors
> 
 >Histone lysine demethylase inhibitors
> 
 >Histone methyltransferase EZH2 inhibitors
> 
 >Histone-arginine methyltransferase inhibitors
> 
 >Histone-lysine methyltransferase inhibitors
> 
 >Hormone antagonists
> 
 >Human herpesvirus 2 vaccines
> 
 >Human papillomavirus vaccines
> 
 >Hyaluronidase inhibitors
> 
 >Hyperpolarization-activated cyclic nucleotide-gated cation channel blockers
> 
 >Hypnotics and Sedatives
> 
 >Hypolipemic agents
> 
 >Hypoxia-inducible factor 1α inhibitors
> 
 >Ileal bile acid transporter inhibitors
> 
 >Immune checkpoint inhibitors
> 
 >Immunoglobulin E inhibitors
> 
 >Immunomodulators
> 
 >Immunoprotective agents
> 
 >Immunostimulants
> 
 >Immunosuppressants
> 
 >Inactivated poliovirus vaccines
> 
 >Indoleamine 2,3-dioxygenase inhibitors
> 
 >Inflammasome inhibitors
> 
 >Influenza vaccines
> 
 >Inhalation anesthetics
> 
 >Inotropics
> 
 >Inotropics, calcium sensitizers
> 
 >Inotropics, negative
> 
 >Insulin mimetics
> 
 >Insulin secretagogues
> 
 >Insulin sensitizers
> 
 >Integrase inhibitors
> 
 >Integrin antagonists
> 
 >Interferon inducers
> 
 >Interleukin 1 inhibitors
> 
 >Interleukin 6 inhibitors
> 
 >Interleukin 6 receptor antagonists
> 
 >Interleukin receptor antagonists
> 
 >Intestinal cleansers
> 
 >Intrauterine contraceptives
> 
 >Intravaginal contraceptives
> 
 >Ion channel blockers
> 
 >Ion channel openers
> 
 >IκB protein kinase inhibitors
> 
 >Janus kinase 1 inhibitors
> 
 >Janus kinase 2 inhibitors
> 
 >Janus kinase inhibitors
> 
 >Japanese encephalitis vaccines
> 
 >Keratolytic agents
> 
 >Kinesin spindle protein inhibitors
> 
 >Kinin antagonists
> 
 >L-type calcium channel blockers
> 
 >Lactate dehydrogenase inhibitors
> 
 >Lantibiotics
> 
 >Laxatives
> 
 >Leishmania amazonensis vaccines
> 
 >Leishmaniasis vaccines
> 
 >Leishmanicides
> 
 >Leucyl-tRNA synthetase inhibitors
> 
 >Leukotriene antagonists
> 
 >Lincosamide antibiotics
> 
 >Lipase inhibitors
> 
 >Lipid-regulating agents
> 
 >Lipoprotein lipase activators
> 
 >Liver X receptor agonists
> 
 >Local anesthetics
> 
 >Lyme disease vaccines
> 
 >Lysine-specific demethylase 1 inhibitors
> 
 >MAP kinase kinase kinase inhibitors
> 
 >MDM2/MDM4 inhibitors
> 
 >Macrolide antibiotics
> 
 >Malaria vaccines
> 
 >Male contraceptives
> 
 >Matrix metalloproteinase inhibitors
> 
 >Matrix metalloproteinase-2 inhibitors
> 
 >Matrix metalloproteinase-9 inhibitors
> 
 >Measles vaccines
> 
 >Measles-mumps-rubella vaccines
> 
 >Measles-mumps-rubella-varicella vaccines
> 
 >Meglitinide site-binding insulin secretagogues
> 
 >Melanin-concentrating hormone receptor 1 antagonists
> 
 >Melanocortin receptor agonists
> 
 >Melanocortin receptor antagonists
> 
 >Melanocortin-4 receptor antagonists
> 
 >Membrane bile acid receptor agonists
> 
 >Membrane transport modulators
> 
 >Meniere disease medications
> 
 >Meningoencephalitis vaccines
> 
 >Metabotropic glutamate receptor agonists
> 
 >Metabotropic glutamate receptor agonists, Group I
> 
 >Metabotropic glutamate receptor agonists, Group II
> 
 >Metabotropic glutamate receptor agonists, Group III
> 
 >Metabotropic glutamate receptor antagonists
> 
 >Metastasis inhibitors
> 
 >Microtubule-disrupting agents
> 
 >Microtubule-stabilizing agents
> 
 >Microtubule-targeting agents
> 
 >Mineralocorticoid receptor antagonists
> 
 >Miotics
> 
 >Mitochondrial inhibitors
> 
 >Mitochondrial trifunctional protein inhibitors
> 
 >Mitoflavoscins
> 
 >Mitogen-activated protein kinase kinase inhibitors
> 
 >Mitogen-activated protein kinase p38 inhibitors
> 
 >Mitogen-activated protein kinase p43 inhibitors
> 
 >Mitoketoscins
> 
 >Mitoriboscins
> 
 >Mitosis modulators
> 
 >Monoacylglycerol lipase inhibitors
> 
 >Monoamine oxidase inhibitors
> 
 >Monoamine reuptake inhibitors
> 
 >Monobactam antibiotics
> 
 >Mood stabilizers
> 
 >Multi-target modulators
> 
 >Mumps vaccines
> 
 >Muraymycins
> 
 >Muscarinic M1 receptor agonists
> 
 >Muscarinic M1 receptor antagonists
> 
 >Muscarinic M2 receptor agonists
> 
 >Muscarinic M2 receptor antagonists
> 
 >Muscarinic M3 receptor agonists
> 
 >Muscarinic M3 receptor antagonists
> 
 >Muscarinic M4 receptor agonists
> 
 >Muscarinic M4 receptor antagonists
> 
 >Muscarinic M5 receptor agonists
> 
 >Muscarinic M5 receptor antagonists
> 
 >Muscarinic agonists
> 
 >Muscarinic antagonists
> 
 >Muscle relaxants
> 
 >Mutation inhibitors
> 
 >Mycobacterium bovis BCG vaccine
> 
 >Mycoplasma hyopneumoniae vaccines
> 
 >Mydriatics
> 
 >Myeloablative agonists
> 
 >Myeloid dendritic cell vaccines
> 
 >N-type calcium channel blockers
> 
 >NEDD8-activating enzyme inhibitors
> 
 >NMDA receptor agonists
> 
 >NMDA receptor antagonists
> 
 >Nanoparticle-coated vaccines
> 
 >Narcotics
> 
 >Nasal vaccines
> 
 >Natriuretics
> 
 >Negative allosteric modulators
> 
 >Nervous system agents
> 
 >Nervous system depressants
> 
 >Neuraminidase inhibitors
> 
 >Neurokinin 1 receptor antagonists
> 
 >Neurokinin 2 receptor antagonists
> 
 >Neurokinin 3 receptor antagonists
> 
 >Neuromuscular agents
> 
 >Neuromuscular blocking agents
> 
 >Neuropeptide Y Y5 receptor antagonists
> 
 >Neuropeptide Y receptor antagonists
> 
 >Neuroprotective agents
> 
 >Neurotransmitter agents
> 
 >Neurotransmitter agonists
> 
 >Neurotransmitter agonists, octopaminergic
> 
 >Neurotransmitter antagonists
> 
 >Neurotransmitter antagonists, excitatory amino acid
> 
 >Neurotransmitter reuptake inhibitors
> 
 >Neutral endopeptidase inhibitors
> 
 >Neutrophil elastase inhibitors
> 
 >Nicotinamide phosphoribosyltransferase inhibitors
> 
 >Nicotinic agonists
> 
 >Nicotinic antagonists
> 
 >Nitric oxide donors
> 
 >Nitric oxide synthase inhibitors
> 
 >Nitric oxide-releasing compounds
> 
 >Non-nucleoside reverse transcriptase inhibitors
> 
 >Nondepolarizing muscle relaxants
> 
 >Nondepolarizing neuromuscular blocking agents
> 
 >Nonnarcotic analgesics
> 
 >Nonsteroidal anti-inflammatory agents
> 
 >Noradrenaline reuptake inhibitors
> 
 >Nuclear export inhibitors
> 
 >Nuclear receptor agonists
> 
 >Nuclear receptor antagonists
> 
 >Nuclease inhibitors
> 
 >Nucleic acid synthesis inhibitors
> 
 >Nucleoside reverse transcriptase inhibitors
> 
 >Nucleotide reverse transcriptase inhibitors
> 
 >Nutrient partitioning agents
> 
 >Oncolytic adenovirus
> 
 >Oncolytic herpes simplex virus
> 
 >Oncolytic virus
> 
 >Ophthalmic agents
> 
 >Opioid receptor antagonists
> 
 >Opioids
> 
 >Opium
> 
 >Oral antidiabetic agents
> 
 >Oral contraceptives
> 
 >Oral vaccines
> 
 >Orexin receptor antagonists
> 
 >Ornithine decarboxylase inhibitors
> 
 >Osmotherapy agents
> 
 >Osmotic dialysis agents
> 
 >Osmotic diuretics
> 
 >Osmotic laxatives
> 
 >Oxytocin receptor antagonists
> 
 >P2 purinoreceptor antagonists
> 
 >P2Y purinoceptor agonists
> 
 >P2Y purinoceptor antagonists
> 
 >P2x purinoreceptor antagonists
> 
 >Papillomavirus vaccines
> 
 >Parainfluenza vaccines
> 
 >Parasiticidal dipping fluids
> 
 >Parasiticides
> 
 >Peptide analogs
> 
 >Peptide antibiotics
> 
 >Peptidomimetics
> 
 >Peripheral vascular disease medications
> 
 >Peroxisome proliferator-activated receptor agonists
> 
 >Peroxisome proliferator-activated receptor α agonists
> 
 >Peroxisome proliferator-activated receptor γ agonists
> 
 >Peroxisome proliferator-activated receptor δ agonists
> 
 >Peroxisome proliferators
> 
 >Pertussis vaccines
> 
 >Pharmaceutical adjuvants
> 
 >Pharmaceutical immune agents
> 
 >Pharmaceutical photosensitizers
> 
 >Pharmacoenhancers
> 
 >Phosphatidylinositol 3-kinase inhibitors
> 
 >Phospho-N-acetylmuramoyl-pentapeptide-transferase inhibitors
> 
 >Phosphodiesterase 1 inhibitors
> 
 >Phosphodiesterase 2 inhibitors
> 
 >Phosphodiesterase 3 inhibitors
> 
 >Phosphodiesterase 4 inhibitors
> 
 >Phosphodiesterase 5 inhibitors
> 
 >Phosphodiesterase 6 inhibitors
> 
 >Phosphodiesterase 7 inhibitors
> 
 >Phosphodiesterase 8 inhibitors
> 
 >Phosphodiesterase 9 inhibitors
> 
 >Phosphodiesterase 10 inhibitors
> 
 >Phosphodiesterase 11 inhibitors
> 
 >Phosphodiesterase inhibitors
> 
 >Phospholipase A2 inhibitors
> 
 >Phospholipase D inhibitors
> 
 >Photoprotectants
> 
 >Photosystem II inhibitors
> 
 >Plague vaccines
> 
 >Plasminogen activator inhibitor 1 inhibitors
> 
 >Platelet aggregation inhibitors
> 
 >Platelet-derived growth factor receptor inhibitors
> 
 >Pneumococcal vaccines
> 
 >Poliovirus vaccines
> 
 >Poly(ADP-ribose) polymerase inhibitors
> 
 >Porcine circovirus vaccines
> 
 >Positive allosteric modulators
> 
 >Potassium channel blockers
> 
 >Potassium channel openers
> 
 >Prebiotics
> 
 >Pregnane X receptor agonists
> 
 >Pro-apoptotic agents
> 
 >Programmed cell death 1 ligand protein inhibitors
> 
 >Programmed cell death protein 1 inhibitors
> 
 >Prokinetic agents
> 
 >Prokinetic agents, gastrokinetic agents
> 
 >Prolyl endopeptidase inhibitors
> 
 >Prostaglandin E2 receptor agonists
> 
 >Prostaglandin antagonists
> 
 >Proteasome inhibitors
> 
 >Protein aggregation inhibitors
> 
 >Protein kinase A inhibitors
> 
 >Protein kinase Akt inhibitors
> 
 >Protein kinase B-Raf inhibitors
> 
 >Protein kinase C inhibitors
> 
 >Protein kinase Raf inhibitors
> 
 >Protein kinase inhibitors
> 
 >Protein synthesis inhibitors
> 
 >Protein tyrosine phosphatase PTP1B inhibitors
> 
 >Proteinase inhibitors
> 
 >Proteolysis-targeting chimeric molecules
> 
 >Proton pump inhibitors
> 
 >Protoporphyrinogen oxidase inhibitors
> 
 >Protozoacides
> 
 >Pseudomonas vaccines
> 
 >Pseudorabies vaccines
> 
 >Psychostimulants
> 
 >Psychotomimetics
> 
 >Psychotropics
> 
 >Pulmonary antihypertensives
> 
 >Purinoceptor agonists
> 
 >Purinoceptor antagonists
> 
 >RNA polymerase inhibitors
> 
 >RNA synthesis inhibitors
> 
 >RNA vaccines
> 
 >ROS1 tyrosine kinase inhibitors
> 
 >Rabies vaccines
> 
 >Radioactive implants
> 
 >Radiomimetics
> 
 >Radiopharmaceuticals
> 
 >Radioprotectants
> 
 >Radiosensitizers, biological
> 
 >Ras inhibitors
> 
 >Ras peptide cancer vaccine
> 
 >Raynaud disease medications
> 
 >Receptor agonists
> 
 >Receptor antagonists
> 
 >Receptor inverse agonists
> 
 >Receptor modulators
> 
 >Receptor tyrosine kinase ALK inhibitors
> 
 >Receptor tyrosine kinase Axl inhibitors
> 
 >Receptor tyrosine kinase EGFR inhibitors
> 
 >Receptor tyrosine kinase inhibitors
> 
 >Regulator of G protein signaling protein inhibitors
> 
 >Renal agents
> 
 >Renal membrane dipeptidase inhibitors
> 
 >Renin inhibitors
> 
 >Renin-angiotensin-aldosterone system inhibitors
> 
 >Renoprotective agents
> 
 >Reproductive control agents
> 
 >Respiratory syncytial virus vaccines
> 
 >Respiratory system agents
> 
 >Retinoic acid receptor agonists
> 
 >Retinoic acid receptor antagonists
> 
 >Retinoid X receptor agonists
> 
 >Retinoid X receptor antagonists
> 
 >Reverse transcriptase inhibitors
> 
 >Rho-associated protein kinase inhibitors
> 
 >Ribonuclease inhibitors
> 
 >Rickettsial vaccines
> 
 >Rotavirus vaccines
> 
 >Rubella vaccines
> 
 >STING agonists
> 
 >STING antagonists
> 
 >Salmonella vaccines
> 
 >Schistosomicides
> 
 >Scopolamine derivatives
> 
 >Secretagogues
> 
 >Selectin antagonists
> 
 >Selective androgen receptor modulators
> 
 >Selective estrogen receptor modulators
> 
 >Selective progesterone receptor modulators
> 
 >Senolytic agents
> 
 >Serine proteinase inhibitors
> 
 >Serine/threonine kinase inhibitors
> 
 >Serotonergic agents
> 
 >Serotonin-dopamine activity modulators
> 
 >Serotonin-noradrenaline reuptake inhibitors
> 
 >Sheddase inhibitors
> 
 >Shigella vaccines
> 
 >Shingles vaccines
> 
 >Sialagogues
> 
 >Sickle cell anemia medications
> 
 >Siderophores
> 
 >Simian AIDS vaccines
> 
 >Skin emollients
> 
 >Skin emollients, cuticle softeners
> 
 >Smallpox vaccines
> 
 >Smooth muscle relaxants
> 
 >Smoothened receptor antagonists
> 
 >Sodium channel agonists
> 
 >Sodium channel blockers
> 
 >Sodium-chloride cotransporter inhibitors
> 
 >Sodium-dependent glucose transporter SGLT2 inhibitors
> 
 >Sodium-iodide symporter inhibitors
> 
 >Sodium-potassium-chloride cotransporter inhibitors
> 
 >Somatostatin receptor agonists
> 
 >Somatostatin receptor antagonists
> 
 >Sphingosine 1-phosphate receptor agonists
> 
 >Sphingosine kinase inhibitors
> 
 >Spinal anesthetics
> 
 >Spirocheticides
> 
 >Spleen tyrosine kinase inhibitors
> 
 >Spliceosome inhibitors
> 
 >Sporicides
> 
 >Squalene synthase inhibitors
> 
 >Squalestatins
> 
 >Steroid 5α-reductase inhibitors
> 
 >Steroid sulfatase inhibitors
> 
 >Steroid synthesis inhibitors
> 
 >Streptococcal vaccines
> 
 >Stromelysin inhibitors
> 
 >Sulfonylurea insulin secretagogues
> 
 >Sunscreens
> 
 >Superoxide dismutase mimetics
> 
 >Sustained-release fungicides
> 
 >Synthetic vaccines
> 
 >Systemic fungicides
> 
 >T cell receptor therapy
> 
 >T-type calcium channel blockers
> 
 >Tachykinin antagonists
> 
 >Telomerase inhibitors
> 
 >Tetracyclic antidepressants
> 
 >Thrombolytics
> 
 >Thromboxane receptor antagonists
> 
 >Thymidylate synthase inhibitors
> 
 >Thyroid hormone receptor agonists
> 
 >Thyroid hormone receptor antagonists
> 
 >Tobacco use cessation agents
> 
 >Tocolytic agents
> 
 >Toll-like receptor 1 agonists
> 
 >Toll-like receptor 2 agonists
> 
 >Toll-like receptor 2 antagonists
> 
 >Toll-like receptor 3 agonists
> 
 >Toll-like receptor 4 agonists
> 
 >Toll-like receptor 4 antagonists
> 
 >Toll-like receptor 5 agonists
> 
 >Toll-like receptor 6 agonists
> 
 >Toll-like receptor 7 agonists
> 
 >Toll-like receptor 8 agonists
> 
 >Toll-like receptor 9 agonists
> 
 >Toll-like receptor 9 antagonists
> 
 >Toll-like receptor 11 agonists
> 
 >Toll-like receptor agonists
> 
 >Toll-like receptor antagonists
> 
 >Tooth desensitizers
> 
 >Topical anesthetics
> 
 >Topical anti-inflammatory agents
> 
 >Topoisomerase I inhibitors
> 
 >Topoisomerase II inhibitors
> 
 >Topoisomerase inhibitors
> 
 >Tranquilizers
> 
 >Tranquilizers, major
> 
 >Tranquilizers, minor
> 
 >Transcription factor Myc inhibitors
> 
 >Transcription factor NF-κB inhibitors
> 
 >Transcription factor STAT3 inhibitors
> 
 >Transcription factor inhibitors
> 
 >Transient receptor potential cation channel TRPM8 agonists
> 
 >Transient receptor potential cation channel TRPM8 antagonists
> 
 >Transport protein inhibitors
> 
 >Trichomonacides
> 
 >Tricyclic antidepressants
> 
 >Trypanosomicides
> 
 >Trypsin inhibitors
> 
 >Tryptophan hydroxylase inhibitors
> 
 >Tuberculosis vaccines
> 
 >Tuberculostatics
> 
 >Tumor necrosis factor inhibitors
> 
 >Tumor-infiltrating lymphocyte therapy
> 
 >Typhoid fever vaccines
> 
 >Tyrosine kinase HER2 inhibitors
> 
 >Tyrosine kinase Src inhibitors
> 
 >Tyrosine kinase inhibitors
> 
 >Tyrosine kinase receptor FLT3 inhibitors
> 
 >Urate transporter 1 inhibitors
> 
 >Uricosuric agents
> 
 >Vaccines
> 
 >Vascular disrupting agents
> 
 >Vascular endothelial growth factor receptor antagonists
> 
 >Vasoconstrictors
> 
 >Vasoconstrictors, arterial
> 
 >Vasoconstrictors, venous
> 
 >Vasodilators
> 
 >Vasodilators, arterial
> 
 >Vasodilators, coronary
> 
 >Vasodilators, peripheral
> 
 >Vasodilators, pulmonary
> 
 >Vasodilators, renal
> 
 >Vasodilators, venous
> 
 >Vasopressin receptor agonists
> 
 >Vasopressin receptor antagonists
> 
 >Venotonic agents
> 
 >Venturicidins
> 
 >Vesicular monoamine transporter inhibitors
> 
 >Viral hepatitis vaccines
> 
 >Viral maturation inhibitors
> 
 >Virosome-based vaccines
> 
 >Virus entry inhibitors
> 
 >Virus-based biopharmaceuticals
> 
 >Vitamin D receptor agonists
> 
 >Vitamin D receptor antagonists
> 
 >Voltage-gated potassium channel activators
> 
 >Voltage-gated potassium channel inhibitors
> 
 >Voltage-gated sodium channel agonists
> 
 >West Nile virus encephalitis vaccines
> 
 >Wound healing promoters
> 
 >Xanthine oxidase inhibitors
> 
 >Yellow fever vaccines
> 
 >c-Jun N-terminal kinase inhibitors
> 
 >mTOR inhibitors
> 
 >mTORC1 inhibitors
> 
 >tRNA synthetase inhibitors
> 
 >α1-Adrenoceptor agonists
> 
 >α1-Adrenoceptor antagonists
> 
 >α2-Adrenoceptor agonists
> 
 >α2-Adrenoceptor antagonists
> 
 >α-Adrenoceptor agonists
> 
 >α-Adrenoceptor antagonists
> 
 >α-Amylase inhibitors
> 
 >α-Glucosidase inhibitors
> 
 >β1-Adrenoceptor agonists
> 
 >β1-Adrenoceptor antagonists
> 
 >β2-Adrenoceptor agonists
> 
 >β2-Adrenoceptor antagonists
> 
 >β3-Adrenoceptor agonists
> 
 >β3-Adrenoceptor antagonists
> 
 >β-Adrenoceptor agonists
> 
 >β-Adrenoceptor antagonists
> 
 >β-Catenin inhibitors
> 
 >β-Hydroxysteroid dehydrogenase inhibitors
> 
 >β-Lactam antibiotics
> 
 >β-Lactamase inhibitors
> 
 >β-Secretase 1 inhibitors
> 
 >β-Secretase 2 inhibitors
> 
 >β-Secretase inhibitors
> 
 >γ-Secretase inhibitors
> 
 >γ-Secretase modulators
> 
 >δ-Opioid receptor agonists
> 
 >δ-Opioid receptor antagonists
> 
 >κ-Opioid receptor agonists
> 
 >κ-Opioid receptor antagonists
> 
 >μ-Opioid receptor agonists
> 
 >μ-Opioid receptor antagonists
> 
 >σ1-Receptor agonists
> 
 >σ1-Receptor antagonists
> 
 >σ-Non-opioid intracellular receptor agonists
> 
 >σ-Receptor antagonists


</details>